In [20]:
from itertools import permutations
from pathlib import Path

import numpy as np

from score_strokes import strokeErrorMatrix
from xmlparse import getXmlScore, loadGeometryBases, loadRef, minXml

In [21]:
# Testing a dynamic programming algorithm

def dynamic(ref_char, ref_data, char_data):
    ref, p_ref, _ = ref_data
    g_data, _, base_data, stroke_sets, _, f_names = char_data
    stroke_priority = permutations(range(0, len(ref)-1))
    for (geometry_length, bases, stroke_set, f_name) in zip(g_data, base_data, stroke_sets, f_names):
        strokes, p_strokes = geometry_length
        error_maps = strokeErrorMatrix(strokes, ref, p_strokes, p_ref)
        stroke_maps = []
        heuristic_scores = []
        # Find candidate stroke orders
        for priority in stroke_priority:
            for i in priority:
                stroke_map = np.full(len(strokes), -1)
                smallerror = np.argmin(error_maps[i]) # retrieve smallest error for current archetype stroke
                while(stroke_map[smallerror]!=-1):
                    # change small error so that we do not repeat over indexes that are already taken
                    # just keeps repeating until we land on an index that doesn't already have a value in its place
                    error_maps[i][smallerror] = 10000
                    smallerror = np.argmin(error_maps[i])
                stroke_map[smallerror] = i
            if any(np.array_equal(stroke_map, m) for m in stroke_maps):
                stroke_maps.append(stroke_map)
        # Retrieve scores for each candidate stroke order
        for s in stroke_maps:
            heuristic_xml = minXml(ref_char, bases, stroke_set, np.array(s)+1)
            heuristic_score = getXmlScore(heuristic_xml)
            heuristic_scores.append(heuristic_score)
    return heuristic_scores, stroke_maps

ref_dir = f'{str(Path.home())}/Stylus_Scoring_Generalization/Reference' # archetype directory
data_dir = f'{str(Path.home())}/Stylus_Scoring_Generalization/NewGenes' # gene directory
ref_char = "6709"

ref_data = loadRef(ref_char, ref_dir)
char_data = loadGeometryBases(data_dir, ref_data[2])

print(dynamic(ref_char, ref_data, char_data))

([], [])


In [22]:
def compareHeuristic(algo1, algo2):
    return